In [ ]:
import os
import tempfile
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableAssign
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
embedder = NVIDIAEmbeddings(
    model="nvidia/nv-embedqa-e5-v5"
)

llm = ChatNVIDIA(
    model="openai/gpt-oss-120b", 
    temperature=0.1,
    max_tokens=1024
)

e:\Anaconda\envs\medibot\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:198: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_25708\3149388274.py:5: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  llm = ChatNVIDIA(


In [ ]:
class GuardrailOutput(BaseModel):
    is_relevant: bool = Field(description="Is the question about the document context?")
    reasoning: str = Field(description="Brief reason for the decision")
    
guardrail_system_prompt = """
You are a security filter for a Contract Assistant. 
Analyze the user's question and determine if it is related to contract analysis, 
legal documents, or general information retrieval from a document.

If the question is unrelated (e.g., "tell me a joke", "how is the weather"), 
set is_relevant to false.
Answer based strictly on the context below:
        {context}

        Question: {question}
"""

guardrail_llm = llm.with_structured_output(GuardrailOutput)


guardrail_chain = guardrail_system_prompt | guardrail_llm

In [2]:
!pip install docx2txt

In [ ]:
def process_uploaded_file(file_path):
    # Handle both PDF and DOCX
    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError("Unsupported file format.")
    
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    
    # Vector store setup
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=embedder,
        collection_name="temp_collection"
    )
    return vectorstore.as_retriever()

In [ ]:
def get_rag_chain(retriever):
    prompt = ChatPromptTemplate.from_template("""
    Answer strictly based on the context: {context}
    Question: {question}
    """)
    
    return (
        {"context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)), 
         "question": RunnablePassthrough()}
        | prompt 
        | llm 
        | StrOutputParser()
    )

NameError: name 'path_to_user_file' is not defined